# Virialization

One of the key parts of initializing N-body simulations is to determine not only the particle positions, but also their velocities. Doing so can be a tricky business as constructing systems which have dynamical stability is a mathematically non-trivial task. What's more, in modified gravities / non-newtonian gravities it may be impossible to use many of the standard tools available in the standard dynamical literature. 

In the ``cluster_generator`` library, there are currently 2 implemented approaches for undertaking this task: the first uses the Eddington formula, and the second the Local Maxwellian Approximation. This guide will take you step by step through the necessary background to understand not only how these approaches work, but also which is best for your current project.

---

## Conents

- [Galactic Dynamics](#Galactic-Dynamics)
  - [The Distribution Function](#The-Distribution-Function)
  - [The Collisionless Boltzmann Equation](#The-Collisionless-Boltzmann-Equation)
  - [The Jean's Equation](#The-Jeans-Equation)
  - [The Jean's Theorem](#The-Jeans-Theorem)
- [The Eddington Formula](#The-Eddington-Formula)
- [The Local Maxwellian Approximation](#The-Local-Maxwellian-Approximation)
- [Virialization in ``cluster_generator``](#Virialization-in-cluster_generator)

---

## Galactic Dynamics

Before we begin discussing the nuts and bolts of each method, it is worth reviewing some of the basic tenants of galactic dynamics. For a useful reference and for deeper information, see [Binney and Tremaine](#binney-tremaine).

### The Distribution Function

In describing the necessary mathematics of this task, we are interested in determining the correct positions and velocities to assign to an ensemble of particles in the chosen simulation domain. Generically, it is useful to refer to the system using its **distribution function** $f(\textbf{x},\textbf{v}) $, which acts as the probability distribution function in the phase space of the system
spanned by all possible values of the position and velocity. In principle, if one knows $f(\textbf{x},\textbf{v})$, then the task of generating initial conditions is simple; just sample from the distribution function as needed. Unfortunately, this is often not a viable strategy and a considerable degree of manipulation is required to undertake this task. Nonetheless, the Eddington formula is a simplified case of determining the distribution function which is only viable in spherically symmetric potentials. In more complex cases, approximations or statistical methods are required. The Local Maxwellian approximation is one of these approaches, and aims to determine only moments of the distribution function instead of the distribution function itself.

### The Collisionless Boltzmann Equation

One of the most important results in classical mechanics is the so-called **Liouville Theorem**, which states that given an ensemble of states in a phase space, the local density of states in the neighborhood of any chosen states is constant. In a more intuitive sense, as an "ensemble of systems" flows through phase space with time, it forms an **incompressible flow**. In the context of classical dynamics, we often express this theorem as

$$ \frac{dD}{dt} = [D,\mathcal{H}] + \frac{\partial D}{\partial t} = \frac{\partial D}{\partial t} + \sum_{j=1}^{2N_{\mathrm{dim}}} \frac{\partial D}{\partial x_j} \dot{x}_j,$$

where $\mathcal{H}$ is the systems Hamiltonian, and the brackets indicate the Poisson bracket notation. In the second expression, not that the summation is over coordinates in phase space, i.e. both velocity and space. 

Now, if one thinks of each particle as representing its own "system" subject to the Hamiltonian determined by the global gravitational properties of the system, but ignoring any potential for particle-particle interaction, then instead of the "density of states", we can instead simply use the distribution function. Furthermore, $\dot{\textbf{v}} = -\nabla \Phi$, which yields the equivalent statement

$$ \frac{\partial f}{\partial t} + \textbf{v}\cdot \frac{\partial f}{\partial \textbf{x}} - \nabla \Phi \cdot \frac{\partial f}{\partial \textbf{v}} = 0. $$

This equation is called the **Collisionless Boltzmann Equation**, or the **Vlasov Equation** and is of extreme importance to our current purpose.

The key benefit that comes from this equation is that it can be used to determine moments of the distribution function. It should also be noted that the CBE is applicable <u>regardless of gravitational theory</u> as long as that they're described in Hamiltonian systems (those for which a Lagrangian / Lagrangian density can be consistently obtained in some form). Most importantly, because MOND (in its manifestations as AQUAL and QUMOND) is a classical field theory, it is also satisfied by the CBE.

### The Jean's Equation

By taking moments of the CBE, one can obtain Jean's equation, which has the form

$$ \nu \left\{\frac{\partial \left<v_j\right>}{\partial t} + \sum_i \left<v_i\right> \frac{\partial \left<v_j\right>}{\partial x_i} + \frac{\partial \Phi}{\partial x_j}\right\} + \sum_i \frac{\partial \left(\nu \sigma_{ij}^2\right)}{\partial x_i} = 0, $$

where $\nu$ is the **position space density**, and $\Phi$ is the **force-conveying potential**. We make a special point of noting that it us the <u>force-conveying</u> potential because in multi-potential theories (QUMOND for example), the potential in the Jean's equation must refer to the potential from which the force is derived. The Jean's equation becomes very important in the Local Maxwellian Approach.

### The Jean's Theorem

Not to be confused with the [Jean's Equation](#The-Jean's-Equation), the Jean's Theorem states the following

<div class="alert alert-block alert-info">
<b>Theorem:</b> states that any steady-state solution of the collisionless Boltzmann equation depends on the phase space coordinates only through integrals of motion in the given potential, and conversely any function of the integrals is a steady-state solution.
</div>

Generally, this fact isn't that useful. After all, actually finding the integrals of motion is a troublesome task in and of itself. Fortunately, in spherical symmetry, 4 integrals of motion can be relatively easily provided: $E, \textbf{L}$. In fact, we may even state a special case of the theorem which applies in spherically symmetric systems:


<div class="alert alert-block alert-info">
<b>Theorem:</b> The distribution function $f(\textbf{x},\textbf{v})$ depends <u>only</u> on the angular momentum and the energy in a spherically symmetric system. When the system is irrotational, the distribution function then reduces to a function of only the energy.
</div>

## The Eddington Formula

Now its time to get down to the business of actually using any of this information. Recalling that $f$ is the distribution function, we may combine this with the poisson equation to obtain

$$ \int f(\textbf{x},\textbf{v}) d^3\textbf{v} = \frac{1}{4\pi G} \nabla^2 \Phi$$

Consider a velocity distribution function $ f(\textbf{v}) $. Then (in Newtonian gravity), the RHS of the equation may be expanded in spherical coordinates to yield a non-trivial differential equation in $ f $.In the case of galaxy clusters, the Jean's Theorem applies, stipulating that $ f $ be a function only of energy and angular momentum. If,as is typical, we assume that the system is irrotational, the angular momentum is constant and the function becomes dependent onlyon the energy. To simplify the notation, we denote the **relative potential** to be $ \Psi = -\Phi + \Phi_0 $ and the relative energyto be $ \mathcal{E} = -E + \Phi_0 = \Psi - \frac{1}{2}v^2 $.  In this case, the equation above may be simplified to the form

$$ \rho(r) = 4\pi \int_0^\Psi f(\mathcal{E})\sqrt{2(\Psi - \mathcal{E})} d\mathcal{E}.$$

If $ \rho $ is considered a function of the relative potential, then one may obtain the equation

$$    \frac{1}{\sqrt{8}\pi}\frac{d\rho}{d\Psi} = \int_0^\Psi \frac{f(\mathcal{E}) d\mathcal{E}}{\sqrt{\Psi-\mathcal{E}}} $$

Blinney and Tremaine note that this is an Abel integral and may be inverted to yield 

$$    f(\mathcal{E}) = \frac{1}{\sqrt{8}\pi^2} \frac{d}{d\mathcal{E}}\int_0^\mathcal{E} \frac{d\rho}{d\Psi} \frac{d\Psi}{\sqrt{\mathcal{E}-\Psi}} $$

This is the approach used in ``cluster_generator`` for the generation of virialized systems when using Newtonian dynamics.

<div class="alert alert-block alert-info">
<b>Note:</b> While this approach is viable in Newtonian gravity, its reliance on the classical poisson equation makes it non-implementable for MOND gravities.
</div>

## The Local Maxwellian Approximation
While the Eddington Formula provides an ideal approach in Newtonian gravity, when using non-newtonian gravity or badly behaved profiles even in Newtonian
gravity, analytical approaches may become non-viable; however, the Local Maxwellian approximation (LMA) can be used to obtain a viable estimate
for the distribution function nonetheless. Consider the Jeans Equation in spherically symmetry:

$$\frac{\partial \rho \sigma_r^2}{\partial r} + \frac{2\rho}{r}\left(\sigma_r^2 -\sigma^2_\theta\right) + \rho \frac{\partial \Phi}{\partial r} = 0.$$

If one also assumes that the stress tensor is entirely isotropic, then the equation may be manipulated to allow one to find the velocity dispersion

$$ \sigma_r^2 = \frac{1}{\rho_h}\int_r^\infty \rho_h \frac{d\Phi}{dr} dr $$

We are then able to assume a gaussian distribution function for the velocities, and thus a maxwellian distribution for the speeds

$$ F(v,r) = 4\pi \left(\frac{1}{2\pi \sigma^2}\right) v^2 \exp\left(\frac{-v^2}{2\bar{v_r}^2}\right) $$


## Virialization in ``cluster_generator``



---

## References

<a name="binney-tremaine"> Binney, J., & Tremaine, S. (2011). Galactic dynamics (Vol. 20). Princeton university press. </a>

---
